In [4]:
import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [5]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [6]:
# PDF 파일 경로 설정
pdf_filepath = './data/콘텐츠분쟁해결_사례.pdf'
# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

In [7]:
# PDF 파일 로드
loader = PyPDFLoader(pdf_filepath)
docs = loader.load()

In [8]:
# 1단계: 문서 분할 설정
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

In [9]:
chunks = pythontext_splitter.split_documents(docs)
print(chunks)

[Document(metadata={'producer': 'Hancom PDF 1.3.0.410', 'creator': 'Hancom PDF 1.3.0.410', 'creationdate': '2011-01-20T18:01:33+09:00', 'title': '제 2절 영국사례', 'moddate': '2011-01-20T18:01:33+09:00', 'pdfversion': '1.4', 'source': './data/콘텐츠분쟁해결_사례.pdf', 'total_pages': 109, 'page': 1, 'page_label': '2'}, page_content='콘텐츠분쟁조정 법리 연구 2부- 타 분쟁조정사례 조사 -'), Document(metadata={'producer': 'Hancom PDF 1.3.0.410', 'creator': 'Hancom PDF 1.3.0.410', 'creationdate': '2011-01-20T18:01:33+09:00', 'title': '제 2절 영국사례', 'moddate': '2011-01-20T18:01:33+09:00', 'pdfversion': '1.4', 'source': './data/콘텐츠분쟁해결_사례.pdf', 'total_pages': 109, 'page': 2, 'page_label': '3'}, page_content='차  례      콘텐츠분쟁조정 법리 연구 제2부 - 타 분쟁조정사례 조사 -타 분쟁조정 위원회 콘텐츠 분쟁해결 사례이지현(드림와이즈) 1. 게임····································································································1가'), Document(metadata={'producer': 'Hancom PDF 1.3.0.410', 'creator': 'Hancom PDF 1.3.0.410', 'creationdate': '2011-01-20T18:01:33+09:00', 'title

In [10]:
# 2단계: 임베딩 모델 설정 
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)

In [11]:
# 벡터 저장
vectorstore = FAISS.from_documents(chunks, pythonembeddings)

In [12]:
# 3단계: 검색기 설정 
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)

In [13]:
# 4단계: LLM 설정 
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)


In [14]:
# 5단계: 법률 자문 프롬프트 작성 
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""


In [15]:
# 6단계: QA 체인 생성 
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                  #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,      #  위에서 설정한 검색기
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=pythonprompt_template,
            input_variables=["context", "question"]
        )
    },  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)

In [16]:
# 7단계: 테스트 질문 작성 
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

In [17]:
for i, q in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {q}")
    print(" 답변 생성 중...")
    
    result = pythonqa_chain.invoke({"query": q})
    answer = result["result"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)


【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 바탕으로 답변을 구성하겠습니다.

1. **사례 기반 답변**:
   - 제시된 사례 1-가-1) 2006년 사례에서는 시스템 오류로 인해 아이템이 소멸된 경우, 계정 명의자가 아니면 복구 처리가 불가하다는 결론이 있었습니다. 계정 명의자가 직접 복구 신청을 해야 한다는 점이 강조되었습니다.
   - 2009년 사례에서는 시스템 오류로 인한 손실이 주장되었으나, 다른 사용자들에게 유사한 불만이 접수되지 않았고, 사용 내역 상 큰 문제가 없다는 이유로 시스템 오류로 보기 어렵다는 결론이 내려졌습니다.

2. **법적 근거 제시**:
   - 게임사의 약관에 따라 계정 명의자만이 복구 요청을 할 수 있다는 규정이 있을 수 있습니다. 따라서 약관을 확인하고, 계정 명의자가 복구 요청을 할 수 있도록 해야 합니다.

3. **판례 참조**:
   - 앞서 언급한 사례들에서 볼 수 있듯이, 시스템 오류로 인한 손실에 대한 복구는 게임사의 약관과 정책에 따라 다르게 처리될 수 있습니다. 특히, 계정 명의자 여부와 시스템 오류의 입증 여부가 중요한 쟁점이 됩니다.

4. **실무 가이드**:
   - **1단계**: 게임사의 약관을 확인하여 복구 요청 절차와 계정 명의자 요건을 파악합니다.
   - **2단계**: 계정 명의자가 복구 요청을 직접 할 수 있도록 조치합니다.
   - **3단계**: 시스템 오류로 인한 손실임을 입증할 수 있는 증거(예: 오류 발생 시점의 스크린샷, 로그 기록 등)를 수집합니다.
   - **4단계**: 게임사에 공식적으로 복구 요청을 제출

In [18]:
# 8단계: 분쟁 유형 분류 함수 ( 선택 )
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"


In [19]:
for i, q in enumerate(pythontest_questions, 1):
    result = classify_dispute_type(q)
    print(result)

게임
이러닝
이러닝
게임
None
